<br>
<h1><center>Can i use your (fast) computer??</center></h1>
<h1><center>A Loop that runs for days</center></h1>
<h1><center>Investment Scenarios : 1,033,505 - Estimated Run Time in Hours:  321.53</center></h1>
<h1><center>Rodrigo Medina</center></h1>
<h1><center>November 2022</center></h1>
<br>

# Index


### 1.Methodology
***

### 2.Implementation
***


<br>
<br>

##### <h1><center> 1. Methodology and Rationale</center></h1>

<left><b>  The loop at the end of this code will evaluate investing in all different permutations of:</b> <br><br>
    - 55 different time lenght periods (from 1 year to 55 years) <br><br>
    - 1,152 different starting dates (from July 1926 to July 2022, in increments of one month) <br><br>
    - 23 different investment strategies which consist in grouping the initial group of 49 industries into different size subgroups<br><br></left>
    
 <left><b>   The possible combinations using the project datasets date restrictions are 1,033,505</b></left><b>
<br><br>

 <p><left>Using a similar rationale in the development of the size factor SMB (Small minus Big) and the value factor HML (High minus Low) in the Fama and French 3 Factor Model, this model explores the existence of a new factor in industry portfolios: low number of firms per industry minus high number of firms by industry, simulation of the market neutral strategy of investing long in industries with low number of firms per industry and investing short in industries with high number of firms per industry provides results that are not significant, however the strategy of only investing long on the industries with low number of firms per industry provides more significant results.
     </left></p><br><br>
     

Calculating Loop Permutations:


In [28]:
investment_scenarios = 0

for h in range(2,25):
    bin_quantity = h
    for i in range(1,56):
        for k in range(0,100,1):
                    end_date_shift = k
                    end_date = 202313 - (end_date_shift*100)
                    investment_time_in_years = i
                    start_date = end_date - (investment_time_in_years*100)
                    for x in range(0,12):
                        start_date = start_date -  1
                        last_two_sd = int(str(start_date)[-2:])
                        end_date = start_date + (investment_time_in_years*100)
                        if end_date > 202207:
                         continue
                        if start_date > 202108:
                         continue
                        if last_two_sd > 13:
                         continue
                        if last_two_sd == 0:
                         continue
                        if start_date < 192607:
                         continue 
                        #print('start date ' , start_date, 'end date ', end_date , 'inv period' , i , 'date dif' , (end_date - start_date), 'k ' ,k , 'bin' ,h ,'x=',x)
                        investment_scenarios += 1

In [29]:
Estimated_secs_per_scenario = 1.12 #prior time estimate per scenario
print('Investment Scenarios for all bins: ' + str(investment_scenarios) + ' - Estimated Run Time in Hours: ' , (round((investment_scenarios*Estimated_secs_per_scenario)/3600,2)))          

Investment Scenarios for all bins: 1033505 - Estimated Run Time in Hours:  321.53


<h1><center> 2. Implementation</center></h1>


In [3]:
import pandas as pd
pd.set_option('display.max_rows', 210)
import yfinance as yf 
import matplotlib.pyplot as plt
import seaborn as sns 
import statsmodels.formula.api as smf 
import scipy.stats as stats
import numpy as np
import datetime
import plotly.express as px
import time

In [4]:
# Reading Fama and French Datasets
DF_Monthly_Rets_ValW = pd.read_csv('C:/DF1 - 49_Ind_P_Monthly_Rets_Val_W.csv',skiprows=0)
DF_Monthly_Rets_EqualW = pd.read_csv('C:/DF2 - 49_Ind_P_Monthly_Rets_EQ_W.csv',skiprows=0)
DF_Monthly_FirmsbyIndustry = pd.read_csv('C:/DF3 - 49_Ind_P_Monthly_Number_of_Firms.csv',skiprows=0)
DF_Monthly_Avg_Firm_Size = pd.read_csv('C:/DF4 - 49_Ind_P_Monthly_Avg_Firm_Size.csv',skiprows=0)

In [5]:
# Removing date column from returns data frame 
DF_Monthly_Rets_ValW_2 = DF_Monthly_Rets_ValW.drop(columns=['Date'])
DF_Monthly_Rets_EqualW_2 = DF_Monthly_Rets_EqualW.drop(columns=['Date'])

In [6]:
# Calculating the mean of all industries for a given period but not taking into
# Account industries in which unaccouted values are expressed as -99.99
DF_Monthly_Rets_ValW_BM = pd.DataFrame()
DF_Monthly_Rets_EqualW_BM = pd.DataFrame()

#Value Weighted Benchmark
DF_Monthly_Rets_ValW_BM['Date'] = DF_Monthly_Rets_ValW['Date']
DF_Monthly_Rets_ValW_BM['Date FF'] = DF_Monthly_Rets_ValW['Date']
DF_Monthly_Rets_ValW_BM['BenchMark_ValW'] = DF_Monthly_Rets_ValW_2.where(DF_Monthly_Rets_ValW_2 != -99.99).mean(axis=1)
DF_Monthly_Rets_ValW_BM['Date'] = pd.to_datetime(DF_Monthly_Rets_ValW_BM['Date FF'], format="%Y%m") 
DF_Monthly_Rets_ValW_BM['Date'] = DF_Monthly_Rets_ValW_BM['Date'].dt.date 
DF_Monthly_Rets_ValW_BM.index = DF_Monthly_Rets_ValW_BM['Date']
DF_Monthly_Rets_ValW_BM = DF_Monthly_Rets_ValW_BM.drop(columns=['Date'])

#Equal Weighted Benchmark
DF_Monthly_Rets_EqualW_BM['Date'] = DF_Monthly_Rets_EqualW['Date']
DF_Monthly_Rets_EqualW_BM['Date FF'] = DF_Monthly_Rets_EqualW['Date']
DF_Monthly_Rets_EqualW_BM['BenchMark_EqualW'] = DF_Monthly_Rets_EqualW_2.where(DF_Monthly_Rets_EqualW_2 != -99.99).mean(axis=1)
DF_Monthly_Rets_EqualW_BM['Date'] = pd.to_datetime(DF_Monthly_Rets_EqualW_BM['Date FF'], format="%Y%m") 
DF_Monthly_Rets_EqualW_BM['Date'] = DF_Monthly_Rets_EqualW_BM['Date'].dt.date 
DF_Monthly_Rets_EqualW_BM.index =DF_Monthly_Rets_EqualW_BM['Date']
DF_Monthly_Rets_EqualW_BM = DF_Monthly_Rets_EqualW_BM.drop(columns=['Date'])

In [7]:
# Benchmark now is shifted one period ahead. Since the benchmark selects all industries invest in them 
# in the current period and accrues the return in next period.

DF_Monthly_Rets_ValW_BM['BenchMark_ValW_Lead_1'] = DF_Monthly_Rets_ValW_BM['BenchMark_ValW'].shift(-1)
DF_Monthly_Rets_EqualW_BM['BenchMark_EqualW_Lead_1'] = DF_Monthly_Rets_EqualW_BM['BenchMark_EqualW'].shift(-1)


In [8]:
# Creating an Industry Name Dictionary 
industry_name_dict = dict()
for i in DF_Monthly_Rets_ValW:
      if i != 'Date':
           industry_name_dict[i] = i

In [9]:
# Creating an Industry Dataframe for each industry combaning information of the 4 initial Fama and French Datasets
industry_dict = dict()
#%% Adding Lead variables, and benchmark
for i in industry_name_dict:  
    df = pd.DataFrame()
    df['Date FF'] = DF_Monthly_Rets_ValW['Date']
    df['Indsury'] = i
    df['Firms by Industry'] = DF_Monthly_FirmsbyIndustry[i]
    df['Avg Firm Size'] = DF_Monthly_Avg_Firm_Size[i]
    df['Ret_ValW'] = DF_Monthly_Rets_ValW[i]
    df['Ret_ValW_Lead1'] = DF_Monthly_Rets_ValW[i].shift(-1)
    df['Ret_EqualW'] = DF_Monthly_Rets_EqualW[i]
    df['Ret_EqualW_lead1'] = DF_Monthly_Rets_EqualW[i].shift(-1)
    industry_dict[i] = df

<h2><center> Analysis in Different Time Periods 1 Million Scenarios - Full Sample Period 1926-2022</center></h2>



In [ ]:
data1 = [] 
data2 = []
data3 = []
data4 = []
data5 = []
data6 = []
data7 = []
data8 = []
data9 = []
data10 = []
data11 = []
data12 = []
data13 = []
data14 = []
data15 = []
data16 = []
data17 = []
data18 = []
data19 = []

investment_scenarios = 0
start_time = time.time()
for h in range(2,25):
    bin_quantity = h
    for i in range(1,56):
        for k in range(0,100,1):
                
                    Ind_Cross_Section_DF = pd.DataFrame()
                    df2 = pd.DataFrame()
                    for j in industry_dict.items():
                        df2 = j[1]
                        Ind_Cross_Section_DF =Ind_Cross_Section_DF.append(df2) 
                    Ind_Cross_Section_DF['Date'] = pd.to_datetime(Ind_Cross_Section_DF['Date FF'], format="%Y%m") 
                    Ind_Cross_Section_DF['Date'] = Ind_Cross_Section_DF['Date'].dt.date       
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.sort_values(by=['Date'])
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Ret_ValW"] != -99.99 ]
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Ret_EqualW"] != -99.99 ]
                    Ind_Cross_Section_DF = Ind_Cross_Section_DF.loc[Ind_Cross_Section_DF["Avg Firm Size"] != -99.99 ]
                    end_date_shift = k
                    end_date = 202213 - (end_date_shift*100)
                    investment_time_in_years = i
                    start_date = end_date - (investment_time_in_years*100)
                  
                    for x in range(0,12):
                        start_date = start_date -  1
                        last_two_sd = int(str(start_date)[-2:])
                        end_date = start_date + (investment_time_in_years*100)
                        
                        if end_date > 202207:
                         continue
                        if start_date > 202107 + (investment_time_in_years*100) :
                         continue
                        if last_two_sd > 13:
                         continue
                        if last_two_sd == 0:
                         continue
                        if start_date < 192607:
                         continue
                        
                        Ind_Cross_Section_DF =  Ind_Cross_Section_DF[(Ind_Cross_Section_DF["Date FF"] > start_date) & (Ind_Cross_Section_DF["Date FF"] <= end_date)]

                        #print('start date ' , start_date, 'end date ', end_date , 'inv period' , i , 'date dif' , (end_date - start_date), 'k ' ,k , 'bin' ,h ,'x=',x)
                        investment_scenarios += 1

                        Ind_Cross_Section_DF['Group_Firms by Industry'] = Ind_Cross_Section_DF.groupby('Date')['Firms by Industry'].transform(lambda x: pd.qcut(x, bin_quantity, labels= False, duplicates = 'drop')) # 
                        # Creating a return dataframe returns are shifted one period ahead.
                        firms_by_industry_q_rets_ValueW = pd.pivot_table(Ind_Cross_Section_DF, values='Ret_ValW_Lead1',index='Date',columns='Group_Firms by Industry',aggfunc=np.mean, fill_value=0)
                        firms_by_industry_q_rets_ValueW['Long Strategy Ret'] = firms_by_industry_q_rets_ValueW[0.0]/100
                        firms_by_industry_q_rets_ValueW['Long Strategy Cumm Ret'] = firms_by_industry_q_rets_ValueW['Long Strategy Ret'].add(1).cumprod()
                        #Adding the Benchmark
                        df_firms_by_industry_q_rets_ValueW = pd.merge(firms_by_industry_q_rets_ValueW ,DF_Monthly_Rets_ValW_BM['BenchMark_ValW_Lead_1'].divide(100),left_index=True,right_index=True,how='left')
                        # Adding the cummulative benchmark return
                        df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Cumm_Ret_Lead_1'] = df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].add(1).cumprod()
                        #Creating Results Dataframe
                        data1.append(bin_quantity)
                        data2.append(firms_by_industry_q_rets_ValueW['Long Strategy Ret'].mean()*12) 
                        data3.append(firms_by_industry_q_rets_ValueW['Long Strategy Ret'].std()*(12**.5))
                        data4.append((firms_by_industry_q_rets_ValueW['Long Strategy Ret'].mean()*12)/(firms_by_industry_q_rets_ValueW['Long Strategy Ret'].std()*(12**.5)))
                        data5.append(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].mean()*12) 
                        data6.append(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].std()*(12**.5)) 
                        data7.append((df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].mean()*12)/(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Lead_1'].std()*(12**.5)))
                        data8.append(firms_by_industry_q_rets_ValueW['Long Strategy Cumm Ret'].iat[-1])
                        data9.append(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Cumm_Ret_Lead_1'].iat[-1])
                        data10.append(start_date)
                        data11.append(investment_time_in_years)
                        data12.append(end_date)
                        data13.append((firms_by_industry_q_rets_ValueW['Long Strategy Cumm Ret'].iat[-1])/(df_firms_by_industry_q_rets_ValueW['BenchMark_ValW_Cumm_Ret_Lead_1'].iat[-1])-1)

                        number_of_bins = pd.DataFrame(data1, columns=['Number of bins / Ranking Categories'])
                        Long_strat_annual_return  = pd.DataFrame(data2,  columns=['Long Strategy Mean Annual Ret'])    
                        Long_strat_standard_deviation  = pd.DataFrame(data3,  columns=['Long Strategy Standard Deviation'])    
                        Long_strat_Return_Risk_Ratio = pd.DataFrame(data4,  columns=['Long Strategy Return Risk Ratio'])    
                        BM_mean_annual_return  = pd.DataFrame(data5,  columns=['Benchmark Annual Ret'])    
                        BM_standard_deviation  = pd.DataFrame(data6,  columns=['Benchmark Standard Deviation Ret']) 
                        BM_Return_Risk_Ratio = pd.DataFrame(data7,  columns=['Benchmark Return Risk Ratio']) 
                        Strat_Return_from_first_to_last_period =     pd.DataFrame(data8,  columns=['Strategy Total Cummulative Return']) 
                        BM_Return_from_first_to_last_period =     pd.DataFrame(data9,  columns=['BenchMark Total Cummulative Return'])

                        start_date_df =     pd.DataFrame(data10,  columns=['Investment Start Date']) 
                        start_date_df['Investment Start Date'] = pd.to_datetime(start_date_df['Investment Start Date'], format="%Y%m") 
                        investment_years_df =     pd.DataFrame(data11,  columns=['Investment Period in Years']) 
                        end_date_df =     pd.DataFrame(data12,  columns=['Investment End Date']) 
                        end_date_df['Investment End Date'] = pd.to_datetime(end_date_df['Investment End Date'], format="%Y%m") 
                        strat_performance_vs_benchmark =  pd.DataFrame(data13,  columns=['Strategy Return Performance vs Benchmark Return'])
                        strat_performance_vs_benchmark['Strategy Return Performance vs Benchmark Return'] = strat_performance_vs_benchmark['Strategy Return Performance vs Benchmark Return'].astype(float).map("{:.2%}".format)
                        Strategy_returns_table = pd.concat([number_of_bins,start_date_df,end_date_df,investment_years_df,strat_performance_vs_benchmark ,Strat_Return_from_first_to_last_period,Long_strat_annual_return,
                                                    Long_strat_standard_deviation,Long_strat_Return_Risk_Ratio,BM_Return_from_first_to_last_period,
                                                    BM_mean_annual_return,BM_standard_deviation,BM_Return_Risk_Ratio  ], axis=1)

    data14.append('Categorizing industries into ' + str(bin_quantity) + ' bins/categories')
    data15.append((Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Strategy Return Performance vs Benchmark Return'].str.rstrip('%').astype(float).mean())/100)
    data16.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Long Strategy Mean Annual Ret'].mean(),5))
    data17.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Long Strategy Standard Deviation'].mean(),5))
    data18.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Benchmark Annual Ret'].mean(),5))
    data19.append(round(Strategy_returns_table[Strategy_returns_table['Number of bins / Ranking Categories']==h]['Benchmark Standard Deviation Ret'].mean(),5))
    
    
    strategy_desc = pd.DataFrame(data14, columns=['Strategy Investing in firms with the lowest industries per firm'])        
    average_overperformance_per_bin_qty = pd.DataFrame(data15, columns=['Average Overperformance of Strategy in ' + str((investment_scenarios)/(len(Strategy_returns_table['Number of bins / Ranking Categories'].unique()))) + ' test Scenarios'])        
    average_overperformance_per_bin_qty['Average Overperformance of Strategy in ' + str((investment_scenarios)/(len(Strategy_returns_table['Number of bins / Ranking Categories'].unique()))) + ' test Scenarios'] = average_overperformance_per_bin_qty['Average Overperformance of Strategy in ' + str((investment_scenarios)/(len(Strategy_returns_table['Number of bins / Ranking Categories'].unique()))) + ' test Scenarios'].astype(float).map("{:.2%}".format)
    
    
    average_strategy_annual_return = pd.DataFrame(data16, columns=['Average Strategy Annualized Return'])
    average_strategy_annual_return['Average Strategy Annualized Return'] = average_strategy_annual_return['Average Strategy Annualized Return'].astype(float).map("{:.3%}".format)
    
    average_strategy_standard_deviation = pd.DataFrame(data17, columns=['Average Strategy Standard Deviation'])
    average_strategy_standard_deviation['Average Strategy Standard Deviation'] = average_strategy_standard_deviation['Average Strategy Standard Deviation'].astype(float).map("{:.3%}".format)
    

    average_benchmark_annual_return = pd.DataFrame(data18, columns=['Average Benchmark Annualized Return'])
    average_benchmark_annual_return['Average Benchmark Annualized Return'] = average_benchmark_annual_return['Average Benchmark Annualized Return'].astype(float).map("{:.3%}".format)
    
    average_benchmark_standard_deviation = pd.DataFrame(data19, columns=['Average Benchmark Standard Deviation'])
    average_benchmark_standard_deviation['Average Benchmark Standard Deviation'] = average_benchmark_standard_deviation['Average Benchmark Standard Deviation'].astype(float).map("{:.3%}".format)
    
    
    
    Total_average_returns_per_strategy_table = pd.concat([ strategy_desc, average_overperformance_per_bin_qty, average_strategy_annual_return,  average_strategy_standard_deviation,average_benchmark_annual_return,average_benchmark_standard_deviation ], axis=1)
end_time = time.time()



In [ ]:
print('Investment Scenarios: ' + str(investment_scenarios)  + ' - Run Time: ' , round(end_time - start_time , 2) , ' seconds')

<br><left><b> Sample Results Dataframe</b><br><br>

In [25]:
Strategy_returns_table
# Displaying Strategy_returns_table.head(10) for partial sample run, full sample simulation dataframe will have 1,033,505 rows

,Number of bins / Ranking Categories,Investment Start Date,Investment End Date,Investment Period in Years,Strategy Return Performance vs Benchmark Return,Strategy Total Cummulative Return,Long Strategy Mean Annual Ret,Long Strategy Standard Deviation,Long Strategy Return Risk Ratio,BenchMark Total Cummulative Return,Benchmark Annual Ret,Benchmark Standard Deviation Ret,Benchmark Return Risk Ratio
0,8,2002-07-01,2022-07-01,20,70.64%,13.167621,0.144147,0.170651,0.844688,7.716596,0.116886,0.168079,0.695424
1,8,2002-06-01,2022-06-01,20,67.17%,13.335651,0.145382,0.170919,0.850592,7.977408,0.119017,0.168161,0.707755
2,8,2002-05-01,2022-05-01,20,73.30%,12.672057,0.143353,0.171039,0.838131,7.312234,0.114930,0.167522,0.686063
3,8,2002-04-01,2022-04-01,20,69.09%,13.597276,0.147403,0.170445,0.864813,8.041517,0.120007,0.166340,0.721457
4,8,2002-03-01,2022-03-01,20,64.98%,13.314274,0.146947,0.170795,0.860369,8.070423,0.120698,0.166665,0.724193
5,8,2002-02-01,2022-02-01,20,51.15%,13.076428,0.146643,0.171154,0.856790,8.651144,0.124639,0.166100,0.750386
6,8,2002-01-01,2022-01-01,20,48.65%,12.444523,0.144666,0.171297,0.844534,8.371898,0.123461,0.166374,0.742070
7,8,1997-12-01,2017-12-01,20,158.87%,16.582581,0.157047,0.178011,0.882235,6.405833,0.106373,0.161085,0.660351
8,8,1997-11-01,2017-11-01,20,160.66%,15.974852,0.155794,0.178296,0.873796,6.128727,0.104547,0.161216,0.648494
9,8,1997-10-01,2017-10-01,20,160.10%,15.687766,0.155526,0.178668,0.870477,6.031400,0.104173,0.161547,0.644848


<br><left><b> Sample Results Dataframe</b><br><br>

In [27]:
Total_average_returns_per_strategy_table
# Displaying Total_average_returns_per_strategy_table for sample run

,Strategy Investing in firms with the lowest industries per firm,Average Overperformance of Strategy in 362.0 test Scenarios,Average Strategy Annualized Return,Average Strategy Standard Deviation,Average Benchmark Annualized Return,Average Benchmark Standard Deviation
0,Categorizing industries into 8 bins/categories,43.01%,14.865%,20.298%,13.254%,17.653%
1,Categorizing industries into 9 bins/categories,35.91%,14.631%,20.573%,13.254%,17.653%
2,Categorizing industries into 10 bins/categories,28.84%,14.331%,21.332%,13.254%,17.653%
